In [127]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [128]:
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))

In [129]:
list_num = 0
print(file_list[list_num])
df = pd.read_csv(file_list[list_num],index_col=[0])

../data/timeseries/AMZN/1002000 - Statement - Consolidated Statements of Operations.csv


In [130]:
df

,2020-03-31,2019-12-31,2019-09-30,2019-06-30,2019-03-31,2018-12-31,2018-09-30,2018-06-30,2018-03-31,2017-12-31,...,2011-03-31,2010-12-31,2010-09-30,2010-06-30,2010-03-31,2009-12-31,2009-09-30,2009-06-30,2009-03-31,2008-12-31
consolidatedstatementsofoperations,,,,,,,,,,,,,,,,,,,,,
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,7.545200e+10,8.743700e+10,6.998100e+10,6.340400e+10,5.970000e+10,7.238300e+10,5.657600e+10,5.288600e+10,5.104200e+10,1.778660e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_productmember,4.184100e+10,5.054300e+10,3.972600e+10,3.585600e+10,3.428300e+10,4.470000e+10,3.374600e+10,3.186400e+10,3.160500e+10,1.185730e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___us-gaap_servicemember,3.361100e+10,3.689400e+10,3.025500e+10,2.754800e+10,2.541700e+10,2.768300e+10,2.283000e+10,2.102200e+10,1.943700e+10,5.929300e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
us-gaap_costofgoodsandservicessold,4.425700e+10,5.397700e+10,4.130200e+10,3.633700e+10,3.392000e+10,4.478600e+10,3.300300e+10,3.063200e+10,3.073500e+10,3.849400e+10,...,7.608000e+09,1.031700e+10,5.786000e+09,4.957000e+09,5.501000e+09,1.897800e+10,4.176000e+09,3.518000e+09,3.741000e+09,1.489600e+10
amzn_fulfillmentexpense,1.153100e+10,1.219300e+10,1.016700e+10,9.271000e+09,8.601000e+09,1.002800e+10,8.275000e+09,7.932000e+09,7.792000e+09,8.974000e+09,...,8.550000e+08,1.090000e+09,6.800000e+08,5.820000e+08,5.460000e+08,2.052000e+09,4.660000e+08,4.090000e+08,4.220000e+08,1.658000e+09
us-gaap_marketingexpense,4.828000e+09,6.171000e+09,4.752000e+09,4.291000e+09,3.664000e+09,4.911000e+09,3.303000e+09,2.901000e+09,2.699000e+09,3.441000e+09,...,3.270000e+08,3.760000e+08,2.410000e+08,2.110000e+08,2.010000e+08,6.800000e+08,1.490000e+08,1.290000e+08,1.280000e+08,4.820000e+08
amzn_technologyandcontentexpense,9.325000e+09,9.739000e+09,9.200000e+09,9.065000e+09,7.927000e+09,7.669000e+09,7.162000e+09,7.247000e+09,6.759000e+09,6.314000e+09,...,5.790000e+08,5.180000e+08,4.420000e+08,4.080000e+08,3.660000e+08,1.240000e+09,3.150000e+08,2.990000e+08,2.750000e+08,1.033000e+09
us-gaap_generalandadministrativeexpense,1.452000e+09,1.412000e+09,1.348000e+09,1.270000e+09,1.173000e+09,1.117000e+09,1.041000e+09,1.111000e+09,1.067000e+09,1.045000e+09,...,1.330000e+08,1.430000e+08,1.170000e+08,1.130000e+08,9.700000e+07,3.280000e+08,8.300000e+07,7.700000e+07,6.800000e+07,2.790000e+08
us-gaap_otheroperatingincomeexpensenet,-7.000000e+07,NaN,NaN,NaN,5.000000e+06,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
280522000000.0-6.998100e+10-6.340400e+10-5.970000e+10

87437000000.0

In [87]:
(datetime.strptime(df.columns.max(), "%Y-%m-%d")-datetime.strptime(df.columns.min(), "%Y-%m-%d")).days/(365/4)



43.035616438356165